In [1]:
#라이브러리 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from datetime import datetime

%matplotlib inline

In [2]:
plt.rc("font", family="Malgun Gothic")
#마이너스 폰트 깨지는 문제에 대한 대처          
plt.rc("axes", unicode_minus=False)  

In [3]:
#레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
#폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
%config InlineBackend.figure_format = 'retina'

In [4]:
# 한글폰트 사용을 위해 설치
# 아래 모듈을 설치하고 불러오면 별도의 한글폰트 설정이 필요 없습니다.
!pip install koreanize-matplotlib

import koreanize_matplotlib


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
df = pd.read_csv("data/dtg_01.csv") #encoding = "cp949"를 통해 한글 폰트를 에러 없이 가져올 수 있음.
df.shape #이렇게 양이 많은 데이터를 엑셀로 분석한다면 매우 느리고 에러가 발생할 수 있기 때문에 판다스가 유용하다. 

C:\Users\USER\AppData\Local\Temp\ipykernel_11300\3638510082.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/dtg_01.csv") #encoding = "cp949"를 통해 한글 폰트를 에러 없이 가져올 수 있음.


(1033694, 32)

In [6]:
df.head(3)

,단말ID,차대번호,트립시작일,정보발생일,누적주행거리,일일주행거리,차량속도,RPM,브레이크상태,차량위치X,...,운전자코드,메시지생성일자,메시지생성시간,응답코드,데이터길이,수집유형,DTG정보수,차량코드,차량번호,Column1
0,1229496162,KMJKG18KPGC011086,2022-11-01 9:12:13,2022-11-01 9:11:48,1045469,0,0,125,1,0,...,9179345,2022-11-01,91245,0,4924,C,60,15,충남72아3043,NaN
1,1229496162,KMJKG18KPGC011086,2022-11-01 9:12:13,2022-11-01 9:11:51,1045469,0,0,275,0,0,...,9179345,2022-11-01,91245,0,4924,C,60,15,충남72아3043,NaN
2,1229496162,KMJKG18KPGC011086,2022-11-01 9:12:13,2022-11-01 9:11:52,1045469,0,0,665,0,0,...,9179345,2022-11-01,91245,0,4924,C,60,15,충남72아3043,NaN


In [32]:
# unique로 중복을 제거한 값을 보고 nunique로 그 개수를 세어본다. 
# ts: trip_start
ts = df["트립시작일"].unique()
ts

<DatetimeArray>
['2022-11-01 09:12:13', '2022-11-01 09:33:48', '2022-11-01 09:44:30',
 '2022-11-01 10:18:50', '2022-11-01 12:55:09', '2022-11-01 15:35:33',
 '2022-11-01 15:45:15', '2022-11-01 16:19:50', '2022-11-01 18:45:30',
 '2022-11-02 07:20:10',
 ...
 '2022-11-30 11:04:58', '2022-11-30 13:37:46', '2022-11-30 13:47:33',
 '2022-11-30 15:37:10', '2022-11-30 16:17:16', '2022-11-30 16:25:31',
 '2022-11-30 16:29:25', '2022-11-30 17:19:52', '2022-11-30 19:40:15',
 '2022-11-30 21:33:10']
Length: 381, dtype: datetime64[ns]

In [33]:
len(ts)

381

In [28]:
ts = df["트립시작일"].unique() #고유한 트립시작일 값을 넣은 trip_start 배열 선언
tlen = len(ts) # 고유한 트립시작일 값의 개수
tt = [[None for i in range(14)] for j in range(tlen)] #비어있는(모든 값이 None값으로 초기화된) trip_table 2차원 배열 선

In [29]:
for i in range(tlen):
    dt=df[df["트립시작일"]==ts[i]].copy() #data_table, 트립시작일 단위로 분류한 각 트립의 테이블(일회용)
    dt=dt.values #일회용 테이블의 2차원 배열화
    last = len(dt)-1 #위에서 만들어진 2차원 배열의 마지막 인덱스
    
    tt[i][0] = dt[0][2] #출발일시, 트립시작일과 동일한 값 적

    tt[i][1] = dt[last][3] #도착일시, 같은 트립시작일 중 가장 마지막의 정보 발생일

    tt[i][2] = dt[0][22] #운전자명, 운전자 코드 값

    tt[i][3] = dt[0][11] #DTG상태, 기기상태값

    tt[i][4] = int((datetime.strptime(dt[last][3],"%Y-%m-%d %H:%M:%S") - datetime.strptime(dt[0][2],"%Y-%m-%d %H:%M:%S")).total_seconds())
    #운행시간, (도착일시-트립시작일) 계산, 초단위로 변환 후 int 형으로 변환

    tt[i][5] = dt[last][4]-dt[0][4] #운행거리, (도착일시 시점의 누적주행거리 - 트립시작일 시점의 누적주행거리)

    tt[i][6] = 0 #최고속도, 초기화 후 반복문을 통해 max값 추출
    for j in range(last+1):
        if dt[j][6]>tt[i][6]:
            tt[i][6] = dt[j][6]

    sum = 0 
    for j in range(last+1):
        sum += dt[j][6]
        tt[i][7] = round(sum/(last+1),1) #평균속도, sum 초기화 후 반복문을 통해 

    tt[i][8] = 0 #최고RPM, 초기화 후 반복문을 통해 max값 추출
    for j in range(last+1):
        if dt[j][7]>tt[i][8]:
            tt[i][8] = dt[j][7]

    sum = 0 
    for j in range(last+1):
        sum += dt[j][7]
        tt[i][9] = round(sum/(last+1),1) #평균RPM, sum 초기화 후 반복문을 통해 평균값 계산

    tt[i][10] = dt[0][14] #최소전압 계산
    for j in range(last+1):
        if dt[j][14]<tt[i][10]:
            tt[i][10] = dt[j][14]

    tt[i][11] = 0 #최대전압 계산
    for j in range(last+1):
        if dt[j][14]>tt[i][11]:
            tt[i][11] = dt[j][14]

    tt[i][12] = 0 #과속시간, (과속한 후의 정보발생일 - 과속한 정보발생일), 초단
    for j in range(last+1):
        if dt[j][6]>110:
            tt[i][12] += (datetime.strptime(dt[j+1][3],"%Y-%m-%d %H:%M:%S") - datetime.strptime(dt[j][3],"%Y-%m-%d %H:%M:%S")).total_seconds()
            tt[i][12] = int(tt[i][12]) #int형으로 변환
            
    tt[i][13] = dt[last][4] #해당 트립에서의 최종 누적주행거리값


In [32]:

print(tt)

[['2022-11-01 9:12:13', '2022-11-01 9:15:31', 9179345, 0, 198, 0, 0, 0.0, 722, 498.7, 220, 285, 0, 1045469], ['2022-11-01 9:33:48', '2022-11-01 9:36:48', 9179345, 0, 180, 0, 9, 1.2, 765, 497.2, 205, 285, 0, 1045469], ['2022-11-01 9:44:30', '2022-11-01 10:05:55', 9179345, 0, 1285, 18, 102, 49.1, 1749, 873.1, 200, 285, 0, 1045487], ['2022-11-01 10:18:50', '2022-11-01 11:48:53', 9179345, 0, 5403, 110, 111, 73.0, 1963, 1140.8, 185, 285, 5, 1045597], ['2022-11-01 12:55:09', '2022-11-01 14:36:43', 9179345, 0, 6094, 110, 109, 64.8, 1889, 1054.0, 205, 285, 0, 1045707], ['2022-11-01 15:35:33', '2022-11-01 15:36:00', 9179345, 0, 27, 0, 10, 5.1, 755, 497.2, 200, 285, 0, 1045707], ['2022-11-01 15:45:15', '2022-11-01 16:07:11', 9179345, 0, 1316, 18, 109, 48.0, 1822, 950.3, 185, 285, 0, 1045725], ['2022-11-01 16:19:50', '2022-11-01 18:34:22', 9179345, 0, 8072, 110, 111, 48.8, 1929, 960.7, 205, 285, 11, 1045835], ['2022-11-01 18:45:30', '2022-11-01 20:21:15', 9179345, 0, 5745, 111, 111, 68.8, 1889, 1

In [33]:
col_name = ['출발일시', '도착일시', '운전자명', 'DTG 상태', '운행시간', '운행거리', '최고속도', '평균속도', '최고RPM', '평균RPM', '최소전압', '최대전압','과속시간', '누적주행거리']
tt = pd.DataFrame(tt, columns = col_name)

In [41]:
print(tt)

                    출발일시                 도착일시     운전자명  DTG 상태  운행시간  운행거리   
0     2022-11-01 9:12:13   2022-11-01 9:15:31  9179345       0   198     0  \
1     2022-11-01 9:33:48   2022-11-01 9:36:48  9179345       0   180     0   
2     2022-11-01 9:44:30  2022-11-01 10:05:55  9179345       0  1285    18   
3    2022-11-01 10:18:50  2022-11-01 11:48:53  9179345       0  5403   110   
4    2022-11-01 12:55:09  2022-11-01 14:36:43  9179345       0  6094   110   
..                   ...                  ...      ...     ...   ...   ...   
376  2022-11-30 16:25:31  2022-11-30 16:25:54  6987201       0    23     0   
377  2022-11-30 16:29:25  2022-11-30 17:10:09  6987201       0  2444    18   
378  2022-11-30 17:19:52  2022-11-30 19:13:00  6987201       0  6788    95   
379  2022-11-30 19:40:15  2022-11-30 21:29:56  6987201       0  6581   112   
380  2022-11-30 21:33:10  2022-11-30 21:36:23  6987201       0   193     0   

     최고속도  평균속도  최고RPM   평균RPM  최소전압  최대전압  과속시간   누적주행거리  
0  

In [43]:
df = pd.DataFrame.from_dict(tt)
df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [7]:
df['정보발생일'] = pd.to_datetime(df['정보발생일'])

# '기기상태' 열의 값을 정수로 변환
df['기기상태'] = df['기기상태'].astype(int)

# 에러 판단을 위한 루프
error_start_time = None
df['error'] = "NaN"  # 초기에 모든 행의 'error' 열은 NaN로 설정

for idx, row in df.iterrows():
    if row['기기상태'] == 11:
        if error_start_time is None:
            error_start_time = row['정보발생일']
        else:
            time_diff = row['정보발생일'] - error_start_time
            if time_diff.total_seconds() >= 900:  # 900초(15분) 이상인 경우 'error'로 판단
                df.loc[idx, 'error'] = True
    else:
        error_start_time = None

# 에러가 연속적으로 유지되는 경우를 찾아 'error' 열에 True로 표시합니다.

# 'error' 열의 변수 종류 확인
error_counts = df['error'].value_counts()

print(df)
print(error_counts)

               단말ID               차대번호                트립시작일   
0        1229496162  KMJKG18KPGC011086   2022-11-01 9:12:13  \
1        1229496162  KMJKG18KPGC011086   2022-11-01 9:12:13   
2        1229496162  KMJKG18KPGC011086   2022-11-01 9:12:13   
3        1229496162  KMJKG18KPGC011086   2022-11-01 9:12:13   
4        1229496162  KMJKG18KPGC011086   2022-11-01 9:12:13   
...             ...                ...                  ...   
1033689  1229496162  KMJKG18KPGC011086  2022-11-30 21:33:10   
1033690  1229496162  KMJKG18KPGC011086  2022-11-30 21:33:10   
1033691  1229496162  KMJKG18KPGC011086  2022-11-30 21:33:10   
1033692  1229496162  KMJKG18KPGC011086  2022-11-30 21:33:10   
1033693  1229496162  KMJKG18KPGC011086  2022-11-30 21:33:10   

                      정보발생일   누적주행거리  일일주행거리  차량속도  RPM  브레이크상태      차량위치X   
0       2022-11-01 09:11:48  1045469       0     0  125       1          0  \
1       2022-11-01 09:11:51  1045469       0     0  275       0          0   
2       2

In [13]:
 # 기기상태 오류인 트립시작일 보기
df_error = df[df["error"]==True]

In [14]:
df_error

,단말ID,차대번호,트립시작일,정보발생일,누적주행거리,일일주행거리,차량속도,RPM,브레이크상태,차량위치X,...,메시지생성일자,메시지생성시간,응답코드,데이터길이,수집유형,DTG정보수,차량코드,차량번호,Column1,error
4718,1229496162,KMJKG18KPGC011086,2022-11-01 10:18:50,2022-11-01 11:08:07,1045545,75,98,1231,0,0,...,2022-11-01,110824,0,4924,C,60,15,충남72아3043,NaN,True
4719,1229496162,KMJKG18KPGC011086,2022-11-01 10:18:50,2022-11-01 11:08:08,1045545,75,97,1215,0,0,...,2022-11-01,110824,0,4924,C,60,15,충남72아3043,NaN,True
4720,1229496162,KMJKG18KPGC011086,2022-11-01 10:18:50,2022-11-01 11:08:09,1045545,75,96,1197,0,0,...,2022-11-01,110824,0,4924,C,60,15,충남72아3043,NaN,True
4721,1229496162,KMJKG18KPGC011086,2022-11-01 10:18:50,2022-11-01 11:08:10,1045545,75,95,1192,0,0,...,2022-11-01,110824,0,4924,C,60,15,충남72아3043,NaN,True
4722,1229496162,KMJKG18KPGC011086,2022-11-01 10:18:50,2022-11-01 11:08:11,1045545,75,94,1185,0,0,...,2022-11-01,110824,0,4924,C,60,15,충남72아3043,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494174,1229496162,KMJKG18KPGC011086,2022-11-15 10:12:11,2022-11-15 10:40:05,1052415,81,0,495,0,0,...,2022-11-15,104007,0,1890,C,23,15,충남72아3043,NaN,True
494175,1229496162,KMJKG18KPGC011086,2022-11-15 10:12:11,2022-11-15 10:40:06,1052415,81,0,497,0,0,...,2022-11-15,104007,0,1890,C,23,15,충남72아3043,NaN,True
494176,1229496162,KMJKG18KPGC011086,2022-11-15 10:12:11,2022-11-15 10:40:07,1052415,81,0,498,0,0,...,2022-11-15,104007,0,1890,C,23,15,충남72아3043,NaN,True
494177,1229496162,KMJKG18KPGC011086,2022-11-15 10:12:11,2022-11-15 10:40:08,1052415,81,0,498,0,0,...,2022-11-15,104007,0,1890,C,23,15,충남72아3043,NaN,True


In [17]:
df_error_len = df_error["트립시작일"].unique()

In [18]:
len(df_error_len)

93

In [19]:
df_error["트립시작일"].unique()

array(['2022-11-01 10:18:50', '2022-11-01 12:55:09',
       '2022-11-01 16:19:50', '2022-11-01 18:45:30', '2022-11-02 8:19:26',
       '2022-11-02 10:48:48', '2022-11-02 13:45:51',
       '2022-11-02 14:20:32', '2022-11-02 17:29:05',
       '2022-11-02 20:49:09', '2022-11-03 7:57:57', '2022-11-03 8:19:08',
       '2022-11-03 11:39:00', '2022-11-03 12:18:58',
       '2022-11-03 14:47:26', '2022-11-03 17:29:49',
       '2022-11-03 20:18:42', '2022-11-04 8:20:35', '2022-11-04 9:55:59',
       '2022-11-04 11:33:00', '2022-11-04 14:11:23',
       '2022-11-04 16:41:29', '2022-11-04 20:49:46',
       '2022-11-04 22:05:41', '2022-11-05 8:14:06', '2022-11-05 14:38:30',
       '2022-11-05 16:37:19', '2022-11-05 18:10:52',
       '2022-11-05 20:47:07', '2022-11-05 22:30:58', '2022-11-06 9:18:00',
       '2022-11-06 11:48:58', '2022-11-06 13:05:10',
       '2022-11-06 14:45:34', '2022-11-06 15:19:51',
       '2022-11-06 17:46:50', '2022-11-07 6:45:02', '2022-11-07 8:36:56',
       '2022-11-07 10:0

In [22]:
df["운전자코드"].unique()

array([9179345,       1, 6987201, 8730587, 8033137], dtype=int64)

In [23]:
df["운전자코드"].nunique()

5

In [40]:
df['트립시작일'] = pd.to_datetime(df['트립시작일'])

# 운전자별로 어떤 날짜에 운전했는지 뽑아서 새로운 DataFrame 생성
driver_date_df = df.groupby('운전자코드')['트립시작일'].apply(list).reset_index()
driver_date_df.columns = ['운전자코드', '트립시작일']

# 결과 출력
print(driver_date_df)

     운전자코드                                              트립시작일
0        1  [2022-11-04 07:10:52, 2022-11-04 07:10:52, 202...
1  6987201  [2022-11-04 19:54:53, 2022-11-04 19:54:53, 202...
2  8033137  [2022-11-19 06:48:17, 2022-11-19 06:48:17, 202...
3  8730587  [2022-11-12 07:27:12, 2022-11-12 07:27:12, 202...
4  9179345  [2022-11-01 09:12:13, 2022-11-01 09:12:13, 202...


In [42]:
driver_date_df.to_csv('운전자코드 트립시작일.csv', index=False, encoding='utf-8-sig')

In [29]:
driver_date_df = df.drop_duplicates(subset=['운전자코드', df['트립시작일'].dt.date])
driver_date_df = driver_date_df[['운전자코드', '트립시작일']].reset_index(drop=True)

# 결과 출력
print(driver_date_df)

TypeError: unhashable type: 'Series'

In [30]:
# '트립시작일' 열을 datetime 형식으로 변환
df['트립시작일'] = pd.to_datetime(df['트립시작일'])

# 운전자별로 트립시작일을 그룹화하고 중복을 제거하여 각 운전자별로 고유한 날짜만 출력
driver_date_df = df.drop_duplicates(subset=['운전자코드', df['트립시작일'].dt.date])
driver_date_df = driver_date_df[['운전자코드', '트립시작일']].reset_index(drop=True)

# 결과 출력
print(driver_date_df)

TypeError: unhashable type: 'Series'

In [31]:
# 운전자별로 트립시작일을 그룹화하고 중복을 제거하여 각 운전자별로 고유한 날짜만 출력
unique_trip_start_dates = df.groupby('운전자코드')['트립시작일'].apply(lambda x: x.drop_duplicates()).reset_index(drop=True)

# 결과 출력
print(unique_trip_start_dates)

0     2022-11-04 07:10:52
1     2022-11-04 08:08:04
2     2022-11-04 08:20:35
3     2022-11-04 09:55:59
4     2022-11-04 11:30:33
              ...        
377   2022-11-03 14:47:26
378   2022-11-03 17:19:19
379   2022-11-03 17:29:49
380   2022-11-03 20:11:51
381   2022-11-03 20:18:42
Name: 트립시작일, Length: 382, dtype: datetime64[ns]


In [38]:
ts['운전자코드'] = ts['운전자코드'].astype(int)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [36]:
df.groupby('운전자코드')['트립시작일']

TypeError: unhashable type: 'Series'